## Load Libs

In [1]:
import os
# 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

os.chdir('/content/drive/MyDrive/open_directory/test/IMAGE')
!pwd

Mounted at /content/drive
/content/drive/.shortcut-targets-by-id/18A-45l2iGPLTzGSEx1Bd0wrE_qi8y2Ai/open_directory/test/IMAGE


In [ ]:
ls

all_train_images.pickle  dataset_3d_fold_0.json  dataset_3d_fold_3.json
data_3d_info.csv         dataset_3d_fold_1.json  dataset_3d_fold_4.json
dataset/                 dataset_3d_fold_2.json  image_pj/


In [2]:
!pip install 'monai[all]'
!pip install kaggle --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 939 kB 5.2 MB/s 
     |████████████████████████████████| 87 kB 9.0 MB/s 
     |████████████████████████████████| 8.4 MB 11.0 MB/s 
     |████████████████████████████████| 316 kB 13.0 MB/s 
     |████████████████████████████████| 17.8 MB 667 kB/s 
     |████████████████████████████████| 251 kB 54.3 MB/s 
     |████████████████████████████████| 31.0 MB 1.3 MB/s 
     |████████████████████████████████| 4.4 MB 49.9 MB/s 
     |████████████████████████████████| 125 kB 79.5 MB/s 
     |████████████████████████████████| 16.6 MB 29.7 MB/s 
     |████████████████████████████████| 70.6 MB 8.7 kB/s 
     |████████████████████████████████| 20.3 MB 94.9 MB/s 
     |████████████████████████████████| 95.3 MB 1.3 MB/s 
     |████████████████████████████████| 54.5 MB 59 kB/s 
     |████████████████████████████████| 15.0 MB 52.3 MB/s 
     |██████████████████████████

In [3]:
# 캐글 환경 
os.environ['KAGGLE_USERNAME'] = 'daldarong'
os.environ['KAGGLE_KEY'] = '34a4da55aa9665a8317bdc6de625793d'
# 이 코드 돌리려면 부엉이씨 파일 받아와야 해서.
# !kaggle kernels output yiheng/3d-solution-with-monai-infer -p /path/to/dest

In [4]:
import gc
from glob import glob
import os
import numpy as np
import pandas as pd
import torch
from torch import nn
from monai.inferers import sliding_window_inference
from monai.data import decollate_batch
from monai.handlers.utils import from_engine
from monai.networks.nets import UNet
from torch.cuda.amp import GradScaler, autocast
from tqdm import tqdm
import json
import pandas as pd

In [5]:
from monai.data import CacheDataset, DataLoader
from monai.transforms import (
    Compose,
    Activations,
    AsDiscrete,
    Activationsd,
    AsDiscreted,
    KeepLargestConnectedComponentd,
    Invertd,
    LoadImage,
    Transposed,
    LoadImaged,
    AddChanneld,
    CastToTyped,
    Lambdad,
    Resized,
    EnsureTyped,
    SpatialPadd,
    EnsureChannelFirstd,
)

In [6]:
ls

3d_data/                 data_3d_info.csv  image_pj/  submission.csv
all_train_images.pickle  dataset/          output/


In [7]:
pd.read_csv('./dataset/uw-madison-gi-tract-image-segmentation/train.csv')

,id,class,segmentation
0,case123_day20_slice_0001,large_bowel,NaN
1,case123_day20_slice_0001,small_bowel,NaN
2,case123_day20_slice_0001,stomach,NaN
3,case123_day20_slice_0002,large_bowel,NaN
4,case123_day20_slice_0002,small_bowel,NaN
...,...,...,...
115483,case30_day0_slice_0143,small_bowel,NaN
115484,case30_day0_slice_0143,stomach,NaN
115485,case30_day0_slice_0144,large_bowel,NaN
115486,case30_day0_slice_0144,small_bowel,NaN


## Prepare meta info.

### Thanks awsaf49, this section refers to:
https://www.kaggle.com/code/awsaf49/uwmgi-2-5d-infer-pytorch

In [8]:
# csv파일 넣으면 id를 통해 정보를 얻어서 case, day, slice 컬럼 만들고 df 생성하는 과정
def get_metadata(row):
    data = row['id'].split('_')
    case = int(data[0].replace('case',''))
    day = int(data[1].replace('day',''))
    slice_ = int(data[-1])
    row['case'] = case
    row['day'] = day
    row['slice'] = slice_
    return row

# image_path를 통해 정보를 얻어서	height, width, case 등 컬럼 만들고 df 생성하는 과정
def path2info(row):
    path = row['image_path']
    data = path.split('/')
    slice_ = int(data[-1].split('_')[1])
    case = int(data[-3].split('_')[0].replace('case',''))
    day = int(data[-3].split('_')[1].replace('day',''))
    width = int(data[-1].split('_')[2])
    height = int(data[-1].split('_')[3])
    row['height'] = height
    row['width'] = width
    row['case'] = case
    row['day'] = day
    row['slice'] = slice_
    return row

In [ ]:
# sub_df = pd.read_csv('uw-madison-gi-tract-image-segmentation/sample_submission.csv')  # 이런 형태로 만들어야한다는 샘플이래. 근데 빈 깡통파일
# sub_df = sub_df.drop(columns=['class','predicted']).drop_duplicates()
# sub_df = sub_df.apply(lambda x: get_metadata(x),axis=1)
# sub_df

,id


In [9]:
sub_df = pd.read_csv('dataset/uw-madison-gi-tract-image-segmentation/sample_submission.csv')
if not len(sub_df):# 비어있으면 (깡통파일이면)
    debug = True
    sub_df = pd.read_csv('dataset/uw-madison-gi-tract-image-segmentation/train.csv')[:1000*3]  # 왜 3000까지만 했을까?
    sub_df = sub_df.drop(columns=['class','segmentation']).drop_duplicates()  # 중복값 처리 3개씩 id 같아서 1개로 줄임 => 총 1000개 됨
else:
    debug = False
    sub_df = sub_df.drop(columns=['class','predicted']).drop_duplicates() # 당황스럽다. 깡통파일이던데?
sub_df = sub_df.apply(lambda x: get_metadata(x),axis=1) # case, day, slice 숫자 적어주는 과정.


debug = True 거치면 get_metadata함수를 통해 case, day, slice 컬럼 생긴다. 원래 중복값만 제거해서 id 컬럼만 남은 sub_df가 이런 모양이었다면, 
|id|
-----
case123_day20_slice_0001
3	case123_day20_slice_0002
6	case123_day20_slice_0003
9	case123_day20_slice_0004
12	case123_day20_slice_0005
...	...
2985	case133_day0_slice_0132
2988	case133_day0_slice_0133
2991	case133_day0_slice_0134
2994	case133_day0_slice_0135
2997	case133_day0_slice_0136

In [10]:
# 이렇게 변한다
sub_df  

,id,case,day,slice
0,case123_day20_slice_0001,123,20,1
3,case123_day20_slice_0002,123,20,2
6,case123_day20_slice_0003,123,20,3
9,case123_day20_slice_0004,123,20,4
12,case123_day20_slice_0005,123,20,5
...,...,...,...,...
2985,case133_day0_slice_0132,133,0,132
2988,case133_day0_slice_0133,133,0,133
2991,case133_day0_slice_0134,133,0,134
2994,case133_day0_slice_0135,133,0,135


In [11]:
import pickle
# load
if debug:
  with open("all_train_images.pickle","rb") as fr:
    paths = pickle.load(fr)

else:
    paths = glob(f'dataset/uw-madison-gi-tract-image-segmentation/test/**/*png',recursive=True)
#     paths = sorted(paths)

In [12]:
path_df = pd.DataFrame(paths, columns=['image_path'])
path_df = path_df.apply(lambda x: path2info(x),axis=1)  # image_path	height	width case day slice 컬럼 만들고 값 넣어주기
path_df.head()

,image_path,height,width,case,day,slice
0,./dataset/uw-madison-gi-tract-image-segmentati...,266,266,15,0,6
1,./dataset/uw-madison-gi-tract-image-segmentati...,266,266,15,0,49
2,./dataset/uw-madison-gi-tract-image-segmentati...,266,266,15,0,99
3,./dataset/uw-madison-gi-tract-image-segmentati...,266,266,15,0,114
4,./dataset/uw-madison-gi-tract-image-segmentati...,266,266,15,0,15


In [13]:
path_df['image_path'][0]  # pickle로 path 가져오면 앞에 ./dataset 이라는 경로가 하나 더 생김.

'./dataset/uw-madison-gi-tract-image-segmentation/train/case15/case15_day0/scans/slice_0006_266_266_1.50_1.50.png'

In [ ]:
path_df['image_path'][0]

'uw-madison-gi-tract-image-segmentation/train/case15/case15_day0/scans/slice_0006_266_266_1.50_1.50.png'

## Produce 3d data list for MONAI DataSet

In [14]:
sub_df.info() # 1000개

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 0 to 2997
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      1000 non-null   object
 1   case    1000 non-null   int64 
 2   day     1000 non-null   int64 
 3   slice   1000 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 39.1+ KB


In [ ]:
path_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38496 entries, 0 to 38495
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   image_path  38496 non-null  object
 1   height      38496 non-null  int64 
 2   width       38496 non-null  int64 
 3   case        38496 non-null  int64 
 4   day         38496 non-null  int64 
 5   slice       38496 non-null  int64 
dtypes: int64(5), object(1)
memory usage: 1.8+ MB


In [15]:
# case, day, slice 기준으로 merge 총 1000개의 행을 가진 데이터 프레임 완성.
test_df = sub_df.merge(path_df, on=['case','day','slice'], how='left')

In [16]:
test_df["case_id_str"] = test_df["id"].apply(lambda x: x.split("_", 2)[0])
test_df["day_num_str"] = test_df["id"].apply(lambda x: x.split("_", 2)[1])
test_df["slice_id"] = test_df["id"].apply(lambda x: x.split("_", 2)[2])

In [ ]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 0 to 999
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           1000 non-null   object
 1   case         1000 non-null   int64 
 2   day          1000 non-null   int64 
 3   slice        1000 non-null   int64 
 4   image_path   1000 non-null   object
 5   height       1000 non-null   int64 
 6   width        1000 non-null   int64 
 7   case_id_str  1000 non-null   object
 8   day_num_str  1000 non-null   object
 9   slice_id     1000 non-null   object
dtypes: int64(5), object(5)
memory usage: 85.9+ KB


In [17]:
for group in test_df.groupby(["case_id_str", "day_num_str"]): # group_by 컬럼명이 자연스럽게 인덱스가 된다.
  print(group[0])

('case123', 'day0')
('case123', 'day20')
('case123', 'day22')
('case133', 'day0')
('case77', 'day0')
('case77', 'day18')
('case77', 'day20')


In [18]:
for group in test_df.groupby(["case_id_str", "day_num_str"]): 
  print(group[1])

                          id  case  day  slice  \
288  case123_day0_slice_0001   123    0      1   
289  case123_day0_slice_0002   123    0      2   
290  case123_day0_slice_0003   123    0      3   
291  case123_day0_slice_0004   123    0      4   
292  case123_day0_slice_0005   123    0      5   
..                       ...   ...  ...    ...   
427  case123_day0_slice_0140   123    0    140   
428  case123_day0_slice_0141   123    0    141   
429  case123_day0_slice_0142   123    0    142   
430  case123_day0_slice_0143   123    0    143   
431  case123_day0_slice_0144   123    0    144   

                                            image_path  height  width  \
288  ./dataset/uw-madison-gi-tract-image-segmentati...     266    266   
289  ./dataset/uw-madison-gi-tract-image-segmentati...     266    266   
290  ./dataset/uw-madison-gi-tract-image-segmentati...     266    266   
291  ./dataset/uw-madison-gi-tract-image-segmentati...     266    266   
292  ./dataset/uw-madison-gi-tract

In [19]:
for group in test_df.groupby(["case_id_str", "day_num_str"]): # case_id_str 그룹 하나, day_num_str 그룹 하나. 7개의 그룹 만들기.

    case_id_str, day_num_str = group[0]
    group_id = case_id_str + "_" + day_num_str  # case123_day0
    group_df = group[1].sort_values("slice_id", ascending=True)
    print(group_df.shape[0])

144
144
144
136
144
144
144


In [20]:
test_data = []  # 빈 리스트 만들고, 

for group in test_df.groupby(["case_id_str", "day_num_str"]): # case_id_str 그룹 하나, day_num_str 그룹 하나. 7개의 그룹 만들기.

    case_id_str, day_num_str = group[0]
    group_id = case_id_str + "_" + day_num_str  # case123_day0
    group_df = group[1].sort_values("slice_id", ascending=True)
    n_slices = group_df.shape[0]  # 144, 144, 144, 136, 144, 144, 144
    group_slices, group_ids = [], []
    for idx in range(n_slices): 
        slc = group_df.iloc[idx]  # 한 장 한 장의 슬라이스
        group_slices.append(slc.image_path) # 슬라이스 image_path를 빈 리스트에 넣는다.
        group_ids.append(slc.id)  # 슬라이스 id를 빈 리스트에 넣는다.
    test_data.append({"image": group_slices, "id": group_ids})  # 위에서 만든 image_path, id의 리스트들을 딕셔너리의 밸류값으로 삼는다.그걸 또 리스트에 어펜드 한다. 이 걸 7번 반복한다. 그럼 리스트 안에 7개의 딕셔너리가 생기겠지.

In [ ]:
test_data[0]

{'id': ['case123_day0_slice_0001',
  'case123_day0_slice_0002',
  'case123_day0_slice_0003',
  'case123_day0_slice_0004',
  'case123_day0_slice_0005',
  'case123_day0_slice_0006',
  'case123_day0_slice_0007',
  'case123_day0_slice_0008',
  'case123_day0_slice_0009',
  'case123_day0_slice_0010',
  'case123_day0_slice_0011',
  'case123_day0_slice_0012',
  'case123_day0_slice_0013',
  'case123_day0_slice_0014',
  'case123_day0_slice_0015',
  'case123_day0_slice_0016',
  'case123_day0_slice_0017',
  'case123_day0_slice_0018',
  'case123_day0_slice_0019',
  'case123_day0_slice_0020',
  'case123_day0_slice_0021',
  'case123_day0_slice_0022',
  'case123_day0_slice_0023',
  'case123_day0_slice_0024',
  'case123_day0_slice_0025',
  'case123_day0_slice_0026',
  'case123_day0_slice_0027',
  'case123_day0_slice_0028',
  'case123_day0_slice_0029',
  'case123_day0_slice_0030',
  'case123_day0_slice_0031',
  'case123_day0_slice_0032',
  'case123_day0_slice_0033',
  'case123_day0_slice_0034',
  'case1

In [ ]:
group[0], group[1]

(('case77', 'day20'),                           id  case  day  slice  \
 432  case77_day20_slice_0001    77   20      1   
 433  case77_day20_slice_0002    77   20      2   
 434  case77_day20_slice_0003    77   20      3   
 435  case77_day20_slice_0004    77   20      4   
 436  case77_day20_slice_0005    77   20      5   
 ..                       ...   ...  ...    ...   
 571  case77_day20_slice_0140    77   20    140   
 572  case77_day20_slice_0141    77   20    141   
 573  case77_day20_slice_0142    77   20    142   
 574  case77_day20_slice_0143    77   20    143   
 575  case77_day20_slice_0144    77   20    144   
 
                                             image_path  height  width  \
 432  ./dataset/uw-madison-gi-tract-image-segmentati...     266    266   
 433  ./dataset/uw-madison-gi-tract-image-segmentati...     266    266   
 434  ./dataset/uw-madison-gi-tract-image-segmentati...     266    266   
 435  ./dataset/uw-madison-gi-tract-image-segmentati...     266    26

## Prepare Transforms, Dataset, DataLoader

In [21]:
class cfg:
    img_size = (224, 224, 80)
    in_channels = 1
    out_channels = 3
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    weights = glob("dataset/large*")
    batch_size = 1
    sw_batch_size = 4
    print(device)

cuda:0


In [22]:
cfg.weights

['dataset/large_unet_fold0_0.9024.pth']

In [23]:
ls

3d_data/                 data_3d_info.csv  image_pj/  submission.csv
all_train_images.pickle  dataset/          output/


In [24]:
test_transforms = Compose(
    [
        LoadImaged(keys="image"), # d, h, w
        AddChanneld(keys="image"), # c, d, h, w
        Transposed(keys="image", indices=[0, 2, 3, 1]), # c, w, h, d
        Lambdad(keys="image", func=lambda x: x / x.max()),
#         SpatialPadd(keys="image", spatial_size=cfg.img_size),  # in case less than 80 slices
        EnsureTyped(keys="image", dtype=torch.float32),
    ]
)

test_ds = CacheDataset(
        data=test_data,
        transform=test_transforms,
        cache_rate=0.0,
        num_workers=2,
    )

test_dataloader = DataLoader(
    test_ds,
    batch_size=cfg.batch_size,
    num_workers=2,
    pin_memory=True,
)

In [25]:
test_transforms

In [26]:
test_ds

In [27]:
test_dataloader

## Prepare Network

In [28]:
model = UNet(
    spatial_dims=3,
    in_channels=cfg.in_channels,
    out_channels=cfg.out_channels,
    channels=(32, 64, 128, 256, 512),
    strides=(2, 2, 2, 2),
    kernel_size=3,
    up_kernel_size=3,
    num_res_units=2,
    act="PRELU",
    norm="BATCH",
    dropout=0.2,
    bias=True,
    dimensions=None,
).to(cfg.device)

## Infer

In [29]:
# ref.: https://www.kaggle.com/stainsby/fast-tested-rle
def rle_encode(img):
  # 무손실 데이터 압축 알고리즘입니다. run 이라는 반복적이고 연속적인 데이터를 줄여 데이터를 압축 합니다 
    """ TBD
    
    Args:
        img (np.array): 
            - 1 indicating mask
            - 0 indicating background
    
    Returns: 
        run length as string formated
    """
    
    pixels = img.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

In [ ]:
# !kaggle datasets download -d yiheng/uw3dweights

100% 136M/136M [00:04<00:00, 40.3MB/s]
100% 136M/136M [00:04<00:00, 29.6MB/s]


In [ ]:
# !unzip 'uw3dweights.zip'

Archive:  uw3dweights.zip
  inflating: large_unet_fold0_0.9024.pth  
  inflating: unet_fold0_0.8789.pth   
  inflating: unet_fold0_0.8932.pth   
  inflating: unet_fold0_0.9108.pth   
  inflating: unet_fold0_new_metric_0.8925.pth  


In [ ]:
ls

3d_data/                     model.h5
data_3d_info.csv             submission.csv
dataset_3d_fold_0.json       unet_fold0_0.8789.pth
dataset_3d_fold_1.json       unet_fold0_0.8932.pth
dataset_3d_fold_2.json       unet_fold0_0.9108.pth
dataset_3d_fold_3.json       unet_fold0_new_metric_0.8925.pth
dataset_3d_fold_4.json       uw3dweights.zip
large_unet_fold0_0.9024.pth  uw-madison-gi-tract-image-segmentation@


In [30]:
outputs = []

post_pred = Compose([
    Activations(sigmoid=True),
    AsDiscrete(threshold=0.5),
])

model.eval()
torch.set_grad_enabled(False)
progress_bar = tqdm(range(len(test_dataloader)))
val_it = iter(test_dataloader)
for itr in progress_bar:
    batch = next(val_it)
    test_inputs = batch["image"].to(cfg.device)

    pred_all = []
    for weights in cfg.weights:
        model.load_state_dict(torch.load(weights))
        pred = sliding_window_inference(test_inputs, cfg.img_size, cfg.sw_batch_size, model)
        pred_all.append(pred)
        # do 4 tta
        for dims in [[2], [3], [2, 3]]:
            flip_pred = sliding_window_inference(torch.flip(test_inputs, dims=dims), cfg.img_size, cfg.sw_batch_size, model)
            flip_pred = torch.flip(flip_pred, dims=dims)
            pred_all.append(flip_pred)
    
    pred_all = torch.mean(torch.stack(pred_all), dim=0)[0]
    pred_all = post_pred(pred_all)
    # c, w, h, d to d, c, h, w
    pred_all = torch.permute(pred_all, [3, 0, 2, 1]).cpu().numpy().astype(np.uint8)
    id_outputs = from_engine(["id"])(batch)[0]

    for test_output, id_output in zip(pred_all, id_outputs):
        id_name = id_output[0]
        lb, sb, st = test_output
        outputs.append([id_name, "large_bowel", rle_encode(lb)])
        outputs.append([id_name, "small_bowel", rle_encode(sb)])
        outputs.append([id_name, "stomach", rle_encode(st)])


100%|██████████| 7/7 [02:13<00:00, 19.02s/it]


In [31]:
outputs

[['case123_day0_slice_0001', 'large_bowel', ''],
 ['case123_day0_slice_0001', 'small_bowel', ''],
 ['case123_day0_slice_0001', 'stomach', ''],
 ['case123_day0_slice_0002', 'large_bowel', ''],
 ['case123_day0_slice_0002', 'small_bowel', ''],
 ['case123_day0_slice_0002', 'stomach', ''],
 ['case123_day0_slice_0003', 'large_bowel', ''],
 ['case123_day0_slice_0003', 'small_bowel', ''],
 ['case123_day0_slice_0003', 'stomach', ''],
 ['case123_day0_slice_0004', 'large_bowel', ''],
 ['case123_day0_slice_0004', 'small_bowel', ''],
 ['case123_day0_slice_0004', 'stomach', ''],
 ['case123_day0_slice_0005', 'large_bowel', ''],
 ['case123_day0_slice_0005', 'small_bowel', ''],
 ['case123_day0_slice_0005', 'stomach', ''],
 ['case123_day0_slice_0006', 'large_bowel', ''],
 ['case123_day0_slice_0006', 'small_bowel', ''],
 ['case123_day0_slice_0006', 'stomach', ''],
 ['case123_day0_slice_0007', 'large_bowel', ''],
 ['case123_day0_slice_0007', 'small_bowel', ''],
 ['case123_day0_slice_0007', 'stomach', ''],

In [ ]:
cfg.weights

['dataset/large_unet_fold0_0.9024.pth']

In [ ]:
post_pred

In [ ]:
submit = pd.DataFrame(data=np.array(outputs), columns=["id", "class", "predicted"])

In [ ]:
submit

,id,class,predicted
0,case123_day0_slice_0001,large_bowel,
1,case123_day0_slice_0001,small_bowel,
2,case123_day0_slice_0001,stomach,
3,case123_day0_slice_0002,large_bowel,
4,case123_day0_slice_0002,small_bowel,
...,...,...,...
2995,case77_day20_slice_0143,small_bowel,
2996,case77_day20_slice_0143,stomach,
2997,case77_day20_slice_0144,large_bowel,
2998,case77_day20_slice_0144,small_bowel,


In [ ]:
# Fix sub error, refers to: https://www.kaggle.com/competitions/uw-madison-gi-tract-image-segmentation/discussion/320541
if not debug:
    sub_df = pd.read_csv('dataset/uw-madison-gi-tract-image-segmentation/sample_submission.csv')
    del sub_df['predicted']
    sub_df = sub_df.merge(submit, on=['id','class'])
    sub_df.to_csv('submission.csv',index=False)
else:
    submit.to_csv('submission.csv', index=False)

In [ ]:
df1 = pd.read_csv('submission.csv')
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         3000 non-null   object
 1   class      3000 non-null   object
 2   predicted  819 non-null    object
dtypes: object(3)
memory usage: 70.4+ KB
